In [7]:
# Required installation
!pip install xgboost

In [39]:
# Required imports
import xgboost as xgb
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [23]:
# Reading data
data = pd.read_csv('C:/Users/goldy/Music/Boosting Techniques/ensemble_boosting/pima-indians-diabetes.csv')
data.head()

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure (mm Hg),Triceps skinfold thickness (mm),2-Hour serum insulin (mu U/ml),Body mass index (weight in kg/(height in m)^2),Diabetes pedigree function,Age,Is Diabetic
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [24]:
col= [ 'Plasma glucose concentration',
       'Diastolic blood pressure (mm Hg)', 'Triceps skinfold thickness (mm)',
       '2-Hour serum insulin (mu U/ml)',
       'Body mass index (weight in kg/(height in m)^2)',
       'Diabetes pedigree function', 'Age']
col

['Plasma glucose concentration',
 'Diastolic blood pressure (mm Hg)',
 'Triceps skinfold thickness (mm)',
 '2-Hour serum insulin (mu U/ml)',
 'Body mass index (weight in kg/(height in m)^2)',
 'Diabetes pedigree function',
 'Age']

In [25]:
# Data have many 0 values which need to be replaced by NaN 
for column in col:
    data[column]=data[column].replace(0,np.nan)

In [26]:
#checking for missing values
data.isna().sum()

Number of times pregnant                            0
Plasma glucose concentration                        5
Diastolic blood pressure (mm Hg)                   35
Triceps skinfold thickness (mm)                   227
2-Hour serum insulin (mu U/ml)                    374
Body mass index (weight in kg/(height in m)^2)     11
Diabetes pedigree function                          0
Age                                                 0
Is Diabetic                                         0
dtype: int64

In [27]:
# imputing the missing values
data['Plasma glucose concentration']=data['Plasma glucose concentration'].fillna(data['Plasma glucose concentration'].mode()[0])
data['Diastolic blood pressure (mm Hg)']=data['Diastolic blood pressure (mm Hg)'].fillna(data['Diastolic blood pressure (mm Hg)'].mode()[0])
data['Triceps skinfold thickness (mm)']=data['Triceps skinfold thickness (mm)'].fillna(data['Triceps skinfold thickness (mm)'].mean())
data['2-Hour serum insulin (mu U/ml)']=data['2-Hour serum insulin (mu U/ml)'].fillna(data['2-Hour serum insulin (mu U/ml)'].mean())
data['Body mass index (weight in kg/(height in m)^2)']=data['Body mass index (weight in kg/(height in m)^2)'].fillna(data['Body mass index (weight in kg/(height in m)^2)'].mean())


In [28]:
data.isna().sum()

Number of times pregnant                          0
Plasma glucose concentration                      0
Diastolic blood pressure (mm Hg)                  0
Triceps skinfold thickness (mm)                   0
2-Hour serum insulin (mu U/ml)                    0
Body mass index (weight in kg/(height in m)^2)    0
Diabetes pedigree function                        0
Age                                               0
Is Diabetic                                       0
dtype: int64

In [29]:
# Seperating feature and target columns
x = data.drop(columns='Is Diabetic')
y= data['Is Diabetic']

In [31]:
# As data set differ alot in magnitude we will scale them
scaler = StandardScaler()
scaled_data = scaler.fit_transform(x)

In [63]:
# Seperate train and test data
x_train,x_test,y_train,y_test= train_test_split(scaled_data,y,test_size=.3, random_state=42)

In [64]:
#fitting data in XGBOOST model
model = XGBClassifier(objective='binary:logistic')
model.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [65]:
# Checking test accuracy
pred = model.predict(x_test)
score = accuracy_score(y_test,pred)
score

0.7272727272727273

In [66]:
# Hyperparameter tuning
grid_params={
    'lerning rate': [1,0.5,0.1,0.01,0.001],
    'max_depth': [3,5,10,20],
    'n_estimators':[10,50,100,200]
}
grid= GridSearchCV(model,grid_params, verbose=3)

In [67]:
grid.fit(x_train,y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[CV] lerning rate=1, max_depth=3, n_estimators=10 ....................
[16:59:09] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { lerning rate } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[CV]  lerning rate=1, max_depth=3, n_estimators=10, score=0.852, total=   0.0s
[CV] lerning rate=1, max_depth=3, n_estimators=10 ....................
[16:59:09] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { lerning rate } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s



[CV] lerning rate=1, max_depth=3, n_estimators=50 ....................
[16:59:09] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { lerning rate } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[CV]  lerning rate=1, max_depth=3, n_estimators=50, score=0.757, total=   0.0s
[CV] lerning rate=1, max_depth=3, n_estimators=50 ....................
[16:59:09] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { lerning rate } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[CV]  lerning rate=1, max_depth=5, n_estimators=50, score=0.759, total=   0.0s
[CV] lerning rate=1, max_depth=5, n_estimators=50 ....................
[16:59:11] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { lerning rate } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[CV]  lerning rate=1, max_depth=5, n_estimators=50, score=0.785, total=   0.1s
[CV] lerning rate=1, max_depth=5, n_estimators=50 ....................
[16:59:11] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { lerning rate } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but sl

[CV]  lerning rate=1, max_depth=10, n_estimators=50, score=0.778, total=   0.1s
[CV] lerning rate=1, max_depth=10, n_estimators=50 ...................
[16:59:12] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { lerning rate } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[CV]  lerning rate=1, max_depth=10, n_estimators=50, score=0.713, total=   0.1s
[CV] lerning rate=1, max_depth=10, n_estimators=50 ...................
[16:59:12] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { lerning rate } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but 

...

[CV]  lerning rate=0.001, max_depth=5, n_estimators=50, score=0.759, total=   0.0s
[CV] lerning rate=0.001, max_depth=5, n_estimators=50 ................
[16:59:35] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { lerning rate } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[CV]  lerning rate=0.001, max_depth=5, n_estimators=50, score=0.785, total=   0.1s
[CV] lerning rate=0.001, max_depth=5, n_estimators=50 ................
[16:59:35] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { lerning rate } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not use

[CV] lerning rate=0.001, max_depth=10, n_estimators=50 ...............
[16:59:37] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { lerning rate } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[CV]  lerning rate=0.001, max_depth=10, n_estimators=50, score=0.713, total=   0.1s
[CV] lerning rate=0.001, max_depth=10, n_estimators=50 ...............
[16:59:37] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { lerning rate } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cas

[CV]  lerning rate=0.001, max_depth=20, n_estimators=50, score=0.806, total=   0.1s
[CV] lerning rate=0.001, max_depth=20, n_estimators=50 ...............
[16:59:38] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { lerning rate } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[CV]  lerning rate=0.001, max_depth=20, n_estimators=50, score=0.731, total=   0.1s
[CV] lerning rate=0.001, max_depth=20, n_estimators=50 ...............
[16:59:39] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { lerning rate } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not u

[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:   30.9s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0, gpu_id=-1,
                                     importance_type='gain',
                                     interaction_constraints='',
                                     learning_rate=0.300000012,
                                     max_delta_step=0, max_depth=6,
                                     min_child_weight=1, missing=nan,
                                     monotone_constraints='()',
                                     n_estimators=100, n_...
                                     objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, subsample=1,
                                     tree_method='exact', validate

In [68]:
grid.best_params_

{'lerning rate': 1, 'max_depth': 3, 'n_estimators': 10}

In [72]:
new_model=XGBClassifier(learning_rate= 1, max_depth= 3, n_estimators= 10)

In [73]:
new_model.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=10, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [75]:
pred_new=new_model.predict(x_test)
score_new= accuracy_score(y_test,pred_new)
score_new

0.7532467532467533

In [ ]:
#Saving Model
filename = 'xgboost_diabetic_model.pickle'
pickle.dump(new_model,open(fiename,'wb'))

loaded_model= pickle.load(open(filename,'rb'))

In [78]:
# Saving Scaler model too
filename_scaler='scaler_model.pickle'
pickle.dump(scaler,open(filename_scaler,'wb'))

scaler_model= pickle.load(open(filename_scaler,'rb'))